# SQL Complexity
This file calculates an SQL complexity score for all questions in the dataset using the **SQompLexity** metric, and saves these stats to a separate database table `features_sql`. Along with this complexity score, several stats are included (also coming from SQompLexity) that keep track of the different expressions and the locations of each in the query.

In [1]:
import base64
import html
import itertools
import json
import os
import re
import subprocess

import database
import pandas as pd
from tqdm import tqdm

In [2]:
# Destination of the sqomplexity CLI tool.
SQOMPLEXITY_COMMAND = ["node", "../sqomplexity/dist/sqomplexity.js"]
# If true, it regenerates the SQompLexity score for all rows, even if it was set already.
REGENERATE = True

In [3]:
# Create two separate connections because one will be reading and one will be writing.
db = database.Database("stackoverflow_mysql_queries")
db_updater = database.Database("stackoverflow_mysql_queries")

In [4]:
def store(function_df, store=None) -> pd.DataFrame:
    """
    Execute and save the function result if the pickle file does not exist.
    If it does exist, the pickle file is loaded.
    :param lambda function_df Function that returns a dataframe.
    :param string store
    """
    if store and os.path.exists(store):
        return pd.read_pickle(store)
    df = function_df()
    if store:
        df.to_pickle(store)
    return df


def base64encode(str) -> str:
    try:
        str = str.encode("ascii")
        str = base64.b64encode(str)
        str = str.decode("ascii")
        return str
    except:
        # Return underscore since apparently we cannot encode it (this makes it a failing query).
        return "Xw=="


def batch_generator(generator, batch_size):
    """
    Create batches for a generator.
    """
    while True:
        batch = list(itertools.islice(generator, batch_size))
        if not batch:
            break
        yield batch


def get_sql_complexity(queries, table, column_callable, id_column, total=None, batch_size=10):
    """
    :param list queries: Expects tuples with (Id, Sql).
    :param str table
    :param callable column_callable: A callable that returns a dictionary with columns and their to be assigned values. It receives a tuple (id, json-payload-from-command).
    """

    def dict_to_update_statement(dict) -> str:
        """
        Converts a dictionary to a part of the UPDATE statement.
        """
        q = [key + " = " + str(value) for (key, value) in dict.items()]
        return ", ".join(q)

    with tqdm(total=total) as pbar:
        for batch in batch_generator(queries, batch_size):
            # Call the Node CLI tool with (multiple) base64encoded queries.
            args = SQOMPLEXITY_COMMAND.copy()
            args.extend((base64encode(q[1]) for q in batch))
            args.extend(["-b", "-a"])
            keys = [i[0] for i in batch]
            try:
                result = subprocess.run(args, encoding="utf-8", capture_output=True, text=True)
            except FileNotFoundError:
                # Command probably too long. Retry with single items.
                if batch_size == 1:
                    # Already running a single item. Throw exception.
                    print("Failed to run command for Id={}".format(", ".join(keys)))
                    raise
                get_sql_complexity(
                    batch,
                    table,
                    column_callable,
                    id_column,
                    total=len(batch),
                    batch_size=1,
                )
                pbar.update(len(batch))
                continue

            try:
                for tuple in zip(keys, json.loads(result.stdout)):
                    db_updater.execute(
                        {
                            "UPDATE": table,
                            "SET": dict_to_update_statement(column_callable(tuple)),
                            "WHERE": "{} = {}".format(id_column, tuple[0]),
                        }
                    )
                    yield tuple
            except ValueError as err:
                print("stdout: {}, stderr: {}".format(result.stdout, result.stderr))
                raise
            pbar.update(len(batch))
        pbar.close()


def extract_sql_from_post_body(row) -> tuple:
    body = row[1]
    if isinstance(body, bytes):
        body = body.decode("utf-8")

    # Replace `table` placeholder that is often used. It is an SQL keyword and the parser doesn't accept it.
    body = re.sub(" table\b", " tableX ", body, flags=re.IGNORECASE)
    body = html.unescape(body)

    # Extract code blocks with SELECT, which should always be there since the database has been prefiltered.
    queries = re.findall(r"<pre><code>SELECT(.*?)<\/code><\/pre>", body, flags=re.DOTALL)
    if not len(queries):
        # Wrong closing of tags, fix anyway.
        queries = re.findall(r"<pre><code>SELECT(.*?)<\/pre><\/code>", body, flags=re.DOTALL)

    if len(queries) <= 0:
        queries = ["_"]

    # Take the first query encountered.
    return (row[0], "SELECT " + queries[0])


def query_sql_complexity_rows(query, table, column_callable, id_column) -> list:
    posts = db.query_generator(query)

    queries = (extract_sql_from_post_body(row) for row in posts)

    total = db.query(query.copy() | {"SELECT": "COUNT(*)"})[0][0]
    scores = get_sql_complexity(queries, table, column_callable, id_column, total=total)

    [s for s in scores]


def tuple_to_features_sql_columns(prefix):
    """
    Create column mappings for Questions and Answers in the `features_sql` table.
    """

    def inner(tuple):
        p = tuple[1]
        columns = {
            prefix + "SQLComplexity": p["complexity"] if p["complexity"] > -1 else "NULL",
            prefix + "SQLParseError": 0 if p["complexity"] > -1 else 1,
        }
        if p["complexity"] > -1:
            per_clause = p["stats"]["expressions_per_clause"]
            per_type = p["stats"]["expressions_per_type"]
            return columns | {
                prefix + "SQLSubqueries": p["stats"]["subqueries"],
                prefix + "SQLIsCyclic": 1 if p["stats"]["is_cyclic"] else 0,
                prefix + "SQLExprInSelect": per_clause["select"],
                prefix + "SQLExprInFrom": per_clause["from"],
                prefix + "SQLExprInJoin": per_clause["join"],
                prefix + "SQLExprInWhere": per_clause["where"],
                prefix + "SQLExprInGroupBy": per_clause["group_by"],
                prefix + "SQLExprInHaving": per_clause["having"],
                prefix + "SQLExprInOrderBy": per_clause["order_by"],
                prefix + "SQLExprInLimit": per_clause["limit"],
                prefix + "SQLExprInOffset": per_clause["offset"],
                prefix + "SQLExprTable": per_type["table"],
                prefix + "SQLExprUnary": per_type["unary"],
                prefix + "SQLExprBinary": per_type["binary"],
                prefix + "SQLExprColumn": per_type["column"],
                prefix + "SQLExprNumber": per_type["number"],
                prefix + "SQLExprAggregation": per_type["aggregation_function"],
                prefix + "SQLExprList": per_type["list"],
                prefix + "SQLExprStar": per_type["star"],
                prefix + "SQLExprFunction": per_type["function"],
                prefix + "SQLExprString": per_type["string"],
                prefix + "SQLExprNull": per_type["null"],
            }
        return columns

    return inner

In [5]:
query_questions = {
    "SELECT": "Id, Body",
    "FROM": "posts INNER JOIN features_sql ON features_sql.QuestionId = posts.Id",
    "WHERE": None if REGENERATE else "features_sql.QuestionSQLComplexity IS NULL AND features_sql.QuestionSQLParseError IS NULL",
}
query_answers = {
    "SELECT": "Id, Body",
    "FROM": "answers INNER JOIN features_sql ON features_sql.AnswerId = answers.Id",
    "WHERE": None if REGENERATE else "features_sql.AnswerSQLComplexity IS NULL AND features_sql.AnswerSQLParseError IS NULL",
}

In [7]:
# Run for questions.
query_sql_complexity_rows(query_questions, "features_sql", tuple_to_features_sql_columns("Question"), "QuestionId")

100%|████████████████████████████████████████████████████████████████████████████| 94925/94925 [24:09<00:00, 65.49it/s]


In [8]:
# Run for answers.
query_sql_complexity_rows(query_answers, "features_sql", tuple_to_features_sql_columns("Answer"), "AnswerId")

100%|████████████████████████████████████████████████████████████████████████████| 88039/88039 [21:01<00:00, 69.77it/s]
